In [1]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-06 15:55:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.38MB/s    in 0.4s    

2024-07-06 15:55:52 (2.38 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:200])  


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
print(f"Length of document is {len(text)} character")

Length of document is 1115394 character


In [8]:
# print unique characters in the entire vocabulary
chars = sorted(list(set(text)), reverse=False)
vocab_size = len(chars)
print(f"size of vocabulary is {vocab_size}\nand unique chars are {''.join(chars)}")

size of vocabulary is 65
and unique chars are 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [10]:
# tokenize
str_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_str = {i:ch for i, ch in enumerate(chars)}
encode = lambda word: [str_to_int[character] for character in word] # takes a string and outputs an interger represnentation for each character in it 
decode = lambda ints: "".join([int_to_str[num] for num in ints]) # takes a list of integers and outputs thier corresponding chars joined in a single string
print(encode("Hii there"))
print(decode(encode("Hii there")))

[20, 47, 47, 1, 58, 46, 43, 56, 43]
Hii there
